In [ ]:
import pandas as pd
import numpy as np
import json
import textdistance as td


In [ ]:
def printUniqueTokens(series):
    unique_series = series.unique()
    token_count = {}
    for a in unique_series:
        tokens = a.split(' ')
        for t in tokens:
            if t not in token_count:
                token_count[t] = 1
            else:
                token_count[t] += 1

    for key, value in sorted(token_count.items(), key=lambda item: item[1]):
        print("%s: %s" % (key, value))

In [6]:
golddupes = pd.read_csv('/data/restaurants_DPL.tsv', delimiter='\t')
dupedict = {}
for i, dupeRow in golddupes.iterrows():
    if dupeRow[0] not in dupedict:
        dupedict[dupeRow[0]] = set()
        dupedict[dupeRow[0]].add(dupeRow[0])
    #dupedict[dupeRow[0]].add(dupeRow[0])
    dupedict[dupeRow[0]].add(dupeRow[1])
    
dupesets = list(dupedict.values())
dupeids = [y for x in dupesets for y in x]

NameError: name 'pd' is not defined

In [4]:
def compareToGold(dfparam):
    recdupes = list(dfparam[dfparam.id.map(len) > 1].id)
    true_positive = []
    false_negative = []
    false_positive = []
    for dupeset in dupesets:
        if dupeset in recdupes:
            true_positive.append(dupeset)
        else:
            false_negative.append(dupeset)
    for recset in recdupes:
        if recset not in dupesets:
            false_positive.append(recset)
    print("True positives: " + str(len(true_positive)))
    print("False negatives: " + str(len(false_negative)))
    print("False positives: " + str(len(false_positive)))
    precision = len(true_positive) / (len(true_positive) + len(false_positive))
    recall = len(true_positive) / (len(true_positive) + len(false_negative))
    fscore = (2*precision*recall) / (precision+recall)
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("Fscore: " + str(fscore))
    return true_positive, false_negative, false_positive